In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical

# Load the ASL Alphabet dataset
data_dir = 'archive/asl_alphabet_train/asl_alphabet_train'  # Replace with the path to your dataset
classes = os.listdir(data_dir)
num_classes = len(classes)

# Initialize lists to store images and their labels
images = []
labels = []

# Load images and labels from the dataset
for class_idx, class_name in enumerate(classes):
    class_dir = os.path.join(data_dir, class_name)
    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)  # Ensure color images
        image = cv2.resize(image, (50, 50))  # Resize the image to a consistent size
        images.append(image)
        labels.append(class_idx)

# Convert lists to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Normalize pixel values to the range [0, 1]
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# One-hot encode the labels
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Load pre-trained ResNet model without top layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(50, 50, 3))

# Add your own custom fully connected layers on top
model = Sequential()
model.add(base_model)  # Add ResNet base
model.add(GlobalAveragePooling2D())  # Global average pooling to reduce spatial dimensions
model.add(Dense(256, activation='relu'))  # Add your custom fully connected layers
model.add(Dense(num_classes, activation='softmax'))  # Output layer with num_classes classes

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model (using your preprocessed data)
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Evaluate the model
accuracy = model.evaluate(X_test, y_test)[1]
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Saving h5 file:
model.save('FinalModel_with_ResNet.h5')


Epoch 1/10
1958/1958 [==============================] - 1767s 896ms/step - loss: 0.3496 - accuracy: 0.8917 - val_loss: 0.1489 - val_accuracy: 0.9529
Epoch 2/10
1958/1958 [==============================] - 1752s 895ms/step - loss: 0.0878 - accuracy: 0.9746 - val_loss: 0.0564 - val_accuracy: 0.9826
Epoch 3/10
1958/1958 [==============================] - 1747s 892ms/step - loss: 0.0685 - accuracy: 0.9806 - val_loss: 0.2588 - val_accuracy: 0.9295
Epoch 4/10
1958/1958 [==============================] - 2689s 1s/step - loss: 0.0498 - accuracy: 0.9865 - val_loss: 0.3071 - val_accuracy: 0.9448
Epoch 5/10
1958/1958 [==============================] - 2086s 1s/step - loss: 0.0396 - accuracy: 0.9891 - val_loss: 0.1095 - val_accuracy: 0.9688
Epoch 6/10
1958/1958 [==============================] - 1788s 913ms/step - loss: 0.0373 - accuracy: 0.9899 - val_loss: 0.0207 - val_accuracy: 0.9927
Epoch 7/10
1958/1958 [==============================] - 1772s 905ms/step - loss: 0.0334 - accuracy: 0.9916 - val